In [1]:
import os
import pytesseract
from pdf2image import convert_from_path
import string 
from nltk.stem.snowball import SnowballStemmer
from googletrans import Translator
import pandas as pd

In [11]:
def File_Content(file_name):
    file_str = ''

    if file_name.endswith('.pdf'):
        pdf_file = convert_from_path(file_name)

        for i in range(len(pdf_file)):
            pdf_file[i].save(f'page{str(i + 1)}.jpg', 'JPEG')   # Saves pages as images in the pdf
            file = f'page{str(i + 1)}.jpg'
            file_str += pytesseract.image_to_string(file)
            # os.remove(file)   # Deletes saved image files
    elif file_name.endswith(('.jpg', '.png')):
        file_str = pytesseract.image_to_string(file_name)
        
    return file_str

In [12]:
file_cont= File_Content('Computer Reading.pdf')

In [13]:
with open("Deneme.txt","a",encoding="utf-8") as file:
    file.write(file_cont)  
    

In [14]:
    with open("Deneme.txt","r+",encoding="utf-8") as file:
        content= file.readlines()
        parag=  file.read()
    

        untitled=""
        for i in content:
            if(i.isupper()==True or (i.istitle()==True)):
                untitled += " "
            else:
                untitled += i + " "
    with open("Computer Reading Son.txt","a",encoding="utf-8") as file1:
        for i in untitled:
            file1.write(i)

In [15]:
import string 
from nltk.tokenize import sent_tokenize
with open("Computer Reading Son.txt","r",encoding="utf-8") as file: 
    contents=file.read()
    content_clear =contents.replace("\n","")
    content_clear = content_clear.replace("/n","")
    
    def punctuation_clear(text):
        result = ""
        for i in text:
            if( i not in string.punctuation or i == " " ):
                result+= i 
            else:
                result+= ""
        return result
    def punctuation_clear_list(text):
        result = ""
        for i in text:
            for a in i:
                if( a not in string.punctuation or a == "." or a == " " ):
                    result+= a 
            
        return result
    content_clears = punctuation_clear(content_clear)
    content_split_sentence = punctuation_clear_list(content_clear).split(".")
    content_word_split= set(content_clears.split())
    word_list= list()
    say = 0
    for i in content_word_split:
        if(len(i)>=3):
            word_list.append(i)
            

In [16]:
sentence_list_last = list()
for a in word_list:
    x=0
    for i in content_split_sentence:
        sentence = i.split()
        for j in sentence:
            if a==j:
                sentence_list_last.append(i)
                x+=1
            if x==1:
                break
        if x==1:
            break
            
content_clear_split_last = list()

423


In [17]:
from nltk.stem.snowball import SnowballStemmer
stem_list=[]

for i in word_list:
    stemmer = SnowballStemmer("english")
    stem_list.append(stemmer.stem(i))
#stem_set = set(stem_list)
#stem_list = list(stem_set)
a=  0
for i in stem_list:
    a+=1
print(a)

423


In [22]:
from googletrans import Translator
translator = Translator()
ceviri_list= list()
for i in word_list:
    translation = translator.translate(i,dest = 'tr')
    ceviri_list.append(translation.text)

In [19]:
from nltk.corpus import state_union
import nltk
from nltk.tokenize import PunktSentenceTokenizer
code_list = list()
for i in word_list:
        words = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(words)
        code_list.append(tagged[0][1])
        

In [23]:
import pandas as pd
dict = {"Code:":code_list,"Raw":word_list,"Root":stem_list,"Translation":ceviri_list,"Sentence":sentence_list_last}
df= pd.DataFrame(dict)
df.to_csv("Computer Reading Translate.csv")

In [22]:
import os
import pytesseract
from pdf2image import convert_from_path
import string 
from nltk.stem.snowball import SnowballStemmer
from googletrans import Translator
import pandas as pd
import string 
from nltk.tokenize import sent_tokenize
from PyDictionary import PyDictionary
import json

code_list1 = list()
word_list1 = list()
def vocab_cam(file_name1):
    def File_Content(file_name):
        file_str = ''

        if file_name.endswith('.pdf'):
            pdf_file = convert_from_path(file_name)

            for i in range(len(pdf_file)):
                pdf_file[i].save(f'page{str(i + 1)}.jpg', 'JPEG')   # Saves pages as images in the pdf
                file = f'page{str(i + 1)}.jpg'
                file_str += pytesseract.image_to_string(file)
                # os.remove(file)   # Deletes saved image files
        elif file_name.endswith(('.jpg', '.png')):
            file_str = pytesseract.image_to_string(file_name)   
        return file_str
    file_cont= File_Content(file_name1)
    
    with open("Deneme.txt","a",encoding="utf-8") as file:
        file.write(file_cont)  
    with open("Deneme.txt","r+",encoding="utf-8") as file:
        content= file.readlines()
        parag=  file.read()
        untitled=""
        for i in content:
            if(i.isupper()==True or (i.istitle()==True)):
                untitled += " "
            else:
                untitled += i + " "
    with open("Computer Reading Son.txt","a",encoding="utf-8") as file1:
        for i in untitled:
            file1.write(i)
    with open("Computer Reading Son.txt","r",encoding="utf-8") as file:
        contents=file.read()
        content_clear =contents.replace("\n","")
        content_clear = content_clear.replace("/n","")
        def punctuation_clear(text):
            result = ""
            for i in text:
                if( i not in string.punctuation or i == " " ):
                    result+= i 
                else:
                    result+= ""
            return result                
        
        def punctuation_clear_list(text):
            result = ""
            for i in text:
                for a in i:
                    if( a not in string.punctuation or a == "." or a == " " ):
                        result+= a 
            return result
        content_clears = punctuation_clear(content_clear)
        content_split_sentence = punctuation_clear_list(content_clear).split(".")
        content_word_split= set(content_clears.split())
        word_list= list()
        
        for i in content_word_split:
            if(len(i)>=3):
                word_list.append(i)
        say1=0
        for q in word_list:
            say1+=1
            word_list1.append(q)
        print("Kelime Listesi : ",say1)
    
    p=0
    sentence_list_last = list()
    for w in word_list:
        
        num=0
        for r in content_split_sentence:
            sentence = r.split()
            for e in sentence:
                if w==e:
                    sentence_list_last.append(r)
                    num+=1
                if num==1:
                    break
            if num==1:
                break
    say2=0
    for q in sentence_list_last:
        say2+=1
    print("Cümle Listesi : ",say2)            
    content_clear_split_last = list()
    from nltk.stem.snowball import SnowballStemmer
    stem_list=[]

    for i in word_list:
        stemmer = SnowballStemmer("english")
        stem_list.append(stemmer.stem(i))
    #stem_set = set(stem_list)
    #stem_list = list(stem_set)
    say3=0
    for q in stem_list:
        say3+=1
    print("kök Listesi : ",say3)
    from googletrans import Translator
    translator = Translator()
    ceviri_list= list()
    for i in word_list:
        translation = translator.translate(i,dest = 'tr')
        ceviri_list.append(translation.text)
    say4=0
    for q in ceviri_list:
        say4+=1
    print("çeviri Listesi : ",say4)
    from nltk.corpus import state_union
    import nltk
    from nltk.tokenize import PunktSentenceTokenizer
    code_list = list()
    for i in word_list:
        words = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(words)
        code_list.append(tagged[0][1])
    say5=0
    for q in code_list:
        say5+=1
        code_list1.append(q)
    print("kod Listesi : ",say5)
    dictionary = PyDictionary()
    pydictlist = list()
    counter = 0
    for i in code_list:
        try:
            if(i=="NN" or i=="NNS" or i=="NNP" or i == "NNPS"):
                definition=dictionary.meaning(word_list[counter],disable_errors=True)["Noun"][0]
                pydictlist.append(definition)
            elif(i == "VB" or i == "VBD" or i == "VBG" or i == "VBN" or i == "VBP" or i  == "VBZ" ):
                definition=dictionary.meaning(word_list[counter],disable_errors=True)["Verb"][0]
                pydictlist.append(definition)
            else:
                pydictlist.append("None")
        except TypeError:
            pydictlist.append("None")
        except KeyError:
            pydictlist.append("None")
        except ValueError:
            pydictlist.append("None")
        except IndexError:
            pydictlist.append("None")
        counter +=1
    
    dict = {"Code:":code_list,"Raw":word_list,"Root":stem_list,"Translation":ceviri_list,"Eng Translation":pydictlist,"Sentence":sentence_list_last,}
    df= pd.DataFrame(dict)
    df.to_json("Computer Reading Translate.json")
    return df
        




In [24]:
vocab_cam('Computer Reading.pdf')
    

Kelime Listesi :  423
Cümle Listesi :  423
kök Listesi :  423
çeviri Listesi :  423
kod Listesi :  423


,Code:,Raw,Root,Translation,Eng Translation,Sentence
0,NN,account,account,hesap,a record or narrative description of past events,without any delay while the users are logged ...
1,IN,Wide,wide,Geniş,None,One of the newest and most popular services ...
2,VBG,clicking,click,tıklama,move or strike with a noise,clicking the mouse
3,VBG,producing,produc,üreten,bring forth or yield,Computer uses mentioned in this unit include ...
4,NN,‘computer,comput,'bilgisayar,None,the ‘computer on a chip’
...,...,...,...,...,...,...
418,NN,displayed,display,görüntülenen,None,The main device for inputting the data is a t...
419,VBN,referred,refer,sevk,make reference to,Devices that include a computer circuit are c...
420,NN,smart,smart,akıllı,a kind of pain such as that caused by a wound ...,smart cards which are able to store informati...
421,NN,health,health,sağlık,a healthy state of wellbeing free from disease,smart cards which are able to store informati...


In [18]:
from PyDictionary import PyDictionary
dictionary=PyDictionary()
pydictlist= list()
counter = 0

for i in code_list1:
    try:
        if(i=="NN" or i=="NNS" or i=="NNP" or i == "NNPS"):
            definition=dictionary.meaning(word_list1[counter],disable_errors=True)["Noun"][0]
            pydictlist.append(definition)
        elif(i == "VB" or i == "VBD" or i == "VBG" or i == "VBN" or i == "VBP" or i  == "VBZ" ):
            definition=dictionary.meaning(word_list1[counter],disable_errors=True)["Verb"][0]
            pydictlist.append(definition)
        else:
            pydictlist.append("None")
    except TypeError:
        pydictlist.append("None")
    except KeyError:
        pydictlist.append("None")
    except ValueError:
        pydictlist.append("None")
    except IndexError:
        pydictlist.append("None")
    counter +=1

In [20]:
len(pydictlist)

423